# <h1> Project 1

Gradient - based optimization and differentiable programming

Trajectory optimization of vehicle bicycle model

In [2]:
import numpy as np
import torch as t
import torch.nn as nn
from torch import optim
from torch.nn import utils
import math

In [4]:
lf = 100
lr = 100
l = lf + lr
frame = 0.1
dim_input = 4
dim_output = 2
dim_hidden = 10

In [ ]:
class Dynamics(nn.Module):
    def __init__(self):
        super(Dynamics, self).__init__()
    @staticmethod
    def forward(state,input):
        beta = t.atan(lr*t.tan(input[1])/L)
        x_dot = state[3]*t.cos(beta+state[2])
        y_dot = state[3]*t.sin(beta+state[2])
        psi_dot = state[3]*t.sin(beta)/lr
        state[0] = state[0] + x_dot*frame
        state[1] = state[1] + y_dot*frame
        state[2] = state[2] + psi_dot*frame
        state[3] = state[3] + input[0]*frame
        return state

In [ ]:
class Controller(nn.Module):

    def __init__(self, dim_input, dim_hidden, dim_output):
        """
        dim_input: # of system states
        dim_output: # of actions
        dim_hidden: up to you
        """
        super(Controller, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(dim_input, dim_hidden),
            nn.Tanh(),
            nn.Linear(dim_hidden, dim_output),
            # You can add more layers here
            nn.Sigmoid()
        )

    def forward(self, state):
        input = self.network(state)
        return input

In [ ]:
class Simulation(nn.Module):

    def __init__(self, controller, dynamics, T):
        super(Simulation, self).__init__()
        self.state = self.initialize_state()
        self.controller = controller
        self.dynamics = dynamics
        self.T = T
        self.action_trajectory = []
        self.state_trajectory = []

    def forward(self, state):
        self.action_trajectory = []
        self.state_trajectory = []
        for _ in range(T):
            action = self.controller.forward(state)
            state = self.dynamics.forward(state, action)
            self.action_trajectory.append(action)
            self.state_trajectory.append(state)
        return self.error(state)

    @staticmethod
    def initialize_state():
        state = [1., 0.]  # TODO: need batch of initial states
        return t.tensor(state, requires_grad=False).float()

    def error(self, state):
        return state[0]**2 + state[1]**2


